MV IaaS instance for Database

In [ ]:
gcloud compute instances create wwtapi-db \
    --zone=europe-central2-b \
    --machine-type=e2-small \
    --image-family=debian-11 \
    --image-project=debian-cloud \
    --boot-disk-size=10GB \
    --tags=http-server,https-server \
    --address=34.116.249.25

Firewall configuration to allow SQL connection

In [ ]:
gcloud compute firewall-rules create allow-sql \
    --direction=INGRESS \
    --priority=1000 \
    --network=default \
    --action=ALLOW \
    --rules=tcp:3306 \
    --source-ranges=0.0.0.0/0 \
    --target-tags=http-server,https-server

Upload Dockerfile and init.sql to VM

In [ ]:
gcloud compute scp Dockerfile init.sql wwtapi-db:~ --zone=europe-central2-b

Dockerization inside VM

In [ ]:
gcloud compute ssh wwtapi-db --zone=europe-central2-b

In [ ]:
sudo apt-get update
sudo apt-get install -y docker.io
sudo systemctl start docker
sudo systemctl enable docker


API PaaS instance creation

In [ ]:
gcloud services enable run.googleapis.com

gcloud run deploy wwtapi \
    --source https://github.com/redriottank/wwtapi \
    --region=europe-central2 \
    --allow-unauthenticated \
    --platform=managed

API automatized deployment via github

In [ ]:
gcloud beta builds triggers create github \
    --name=wwtapi-trigger \
    --repo-owner=redriottank \
    --repo-name=wwtapi \
    --branch-pattern="^master$" \
    --build-config=cloudbuild.yaml
